In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from nilearn import datasets
from nilearn import image

import nibabel as nb
import os.path as op

# About this tutorial
Performing machine learning on neuroimaging data means we need to be able to load / manipulate / visualize the neural data in the first place.

Here we'll cover how to work with 3D and 4D neural images, with a focus on the following topics:

* Loading neuroimaging data from disk
* Accessing the raw data and its properties
* Doing some simple visualizations of the data (in 3D and over a 4th dimension such as time)

We'll focus on the following packages:

* nibabel
* nilearn

# 3D and 4D niimgs: handling and visualizing

In python, volume data is represented in a common format called "nifty" format. It has the extension `.nii` or `.nii.gz`. There are many packages for deadling with this format in python, though the most common is called `nibabel`.

## Downloading tutorial datasets from Internet

In this tutorial we won't do any analysis, but instead will take a look at the *results* of a previous analysis. 

Nilearn comes with functions that download public data from Internet. These are common or well-known tasks in neuroscience that have been made public by the researchers. These can be accessed with the following command:

```
from nilearn import datasets
datasets.fetch_<tab>
```

You should see a list of possible datasets to download. Let's now retrieve the maps of the MSDL atlas.

**Note that most datasets don't simply return raw data, but an object with metadata included**

In [ ]:
data = datasets.fetch_atlas_msdl()
print(data['description'].decode('utf-8'))

In [ ]:
map_filename = data['maps']
print(map_filename)

Just as before, we can see that this dataset is well-described. It looks like we have the raw activity for the data in the `func` field, while the statistical maps are stored in `tmaps`. Let's take a look at this and visualize the statistic on the brain.

# Loading a Nifti file


## Nifti format structure
Volume files are generally represented in "nifty" format in python. These might represent neural activity, or statistics calculated from neural activity. They are easy to read with the `nibabel` package.

In [ ]:
# Now load the data
brain = nb.load(map_filename)

### Nifti files contain metadata + data
If you look inside `brain`, you'll find a number of methods and attributes in python that let you do different things with it. For example, we could return the transformation matrix to switch coordinate spaces:

In [ ]:
brain.affine

However, here we will focus on the raw data itslef. The data for this volume is stored in the object, and we can access it as a numpy array:

In [ ]:
brain.get_data()[:2]

**Note: the final dimension of a Nifti file is often time, but here it actually represents different functional regions. Make sure that you check the metadata of our dataset to determine this any data you work with.**

In [ ]:
# This is the dimensionality of the volume
# it is 4D, AKA activity in x/y/z space over some final dimension
brain.get_data().shape

With `nilearn`, we can do some quick plotting simply by pointing to the file name. That way we don't have to load in the data manually.

## Visualizing the file

The file contains a 4volume, and we can easily visualize it as a
statistical map. `nilearn` has a number of functions for visualizing neuroimaging maps. These will generally take either a nifti object, or a file path to a nifti object.

**If we wish to plot a single image, we need to choose a moment in time. Here's we'll use the `index_img` function to do this.**

In [ ]:
from nilearn import plotting

In [ ]:
# Here we'll load the nifty object according to the statistical map
# tmap = nb.load(map_filename)

# Then plot it
plotting.plot_stat_map(image.index_img(brain, 3))

# Alternatively, we could simply point it to the filename
# plotting.plot_stat_map(image.index_img(map_filename, 3))

Note that the plotted background image was automatically assumed to be an MNI brain. If you have your own MRI images you can use these as well.

### Manipulating the image

**...with thresholds**

We can also do simple manipulations of the visualization, such as applying a threshold:

In [ ]:
plotting.plot_stat_map(image.index_img(map_filename, 3), threshold=.3)

**...with different views**

If you wish to see a collection of slices across an axis, you can use the `display_mode` argument:

In [ ]:
# For the y-axis
plotting.plot_stat_map(image.index_img(map_filename, 3), threshold=.3, display_mode='y')

**...with more slices**

You can also define the number of slices that are displayed by passing an integer to the `cut_coords` parameter.

In [ ]:
plotting.plot_stat_map(image.index_img(map_filename, 3), threshold=.3,
                       display_mode='z', cut_coords=10)

# Looping on all volumes in a 4D file

If we want to plot all the volumes in this 4D file, we can use iter_img
to loop on them. This is similar to plotting statistical maps as you move down a particular axis of the 3-D brain, but you're looping across time instead.

Then we give a few arguments to plot_stat_map in order to have a more
compact display. We'll plot a single slice of the z-axis over time.

In [ ]:
for ii, img in enumerate(image.iter_img(map_filename)):
    if ii > 5:
        # This causes us to skip after the first 5 plots
        continue

    # img is now an in-memory 3D img
    plotting.plot_stat_map(img, threshold=.1, colorbar=False,
                           cut_coords=(0, 0, 0))
    


# Recap
To recap, neuroimaging images (niimgs as we call them) come in
different flavors:

* 3D images, containing only one brain volume
* 4D images, containing multiple brain volumes.

More details about the input formats in nilearn for 3D and 4D images is
given in the documentation section for nilearn.

Functions accept either 3D or 4D images, and we need to use these with either `nilearn.image.index_img` or `nilearn.image.iter_img`
to break down 4D images into 3D images.

**Note: There are many other types of neuroimaging data (e.g., surfaces, diffusion maps, etc). In general, `nibabel` should be able to read them with `nb.load` if they're in a standard nifti-style format.**